In [ ]:
import pyautogui
from PIL import ImageGrab, Image
import time
import shutil
import os
import easyocr
import ccxt
import re

#변수 초기화
text = ""  
previous_position = "" 
current_position = ""
previous_btc_amount = 0
current_btc_amount = 0
diff_btc = 0 
previous_buy_amount = 0
current_buy_amount = 0
plus_btc_buy = 0
diff_btcc = 0

with open("api.txt") as f:
    lines = f.readlines()
    api_key = lines[0].strip()
    secret = lines[1].strip()

binance = ccxt.binance(config={
    'apiKey': api_key,
    'secret': secret,
    'enableRateLimit': True,
    'options': {
        'defaultType': 'future'
    }
})
markets = binance.load_markets()
symbol = "BTC/USDT"
market = binance.market(symbol)
buy_coin = binance.fapiprivate_post_leverage({
    'symbol': market['id'],
    'leverage': 10 #레버리지 조정 
})

time.sleep(5) #화면 전환 대기 시간

#pyautogui로 좌표 확인 후 작성
while True:
    
    
    x1, y1 = 115, 790           #좌상단 좌표 (y1 값을 y2보다 작게 설정)
    x2, y2 = 429, 1048
    #우하단 좌표 (y2 값을 y1보다 크게 설정)


    screen_width, screen_height = pyautogui.size()
    x1 = max(0, min(x1, screen_width))
    y1 = max(0, min(y1, screen_height))
    x2 = max(0, min(x2, screen_width))
    y2 = max(0, min(y2, screen_height))
    captured_image = ImageGrab.grab(bbox=(x1, y1, x2, y2))

    folder_name = 'coin'
    desktop_path = os.path.join(os.path.expanduser('~'), 'Desktop')
    custom_folder_path = os.path.join(desktop_path, folder_name)
    captured_image_path = os.path.join(custom_folder_path, 'captured_image.png')
    captured_image.save(captured_image_path)

    coin_folder_path = os.path.join(desktop_path, 'coin')
    image_files = [f for f in os.listdir(coin_folder_path) if f.lower().endswith(('.png'))]
    if image_files:
        first_image_path = os.path.join(coin_folder_path, image_files[0])
        try:
            reader = easyocr.Reader(['en'])  
            result = reader.readtext(first_image_path)
            texts = [item[1] for item in result]  
            text = '\n'.join(texts)  
            print(text)
        except Exception as e:
            print("이미지 error:", str(e))
    
    
    if "btc" in text.lower() or "bic" in text.lower():
        
        
        previous_btc_amount = current_btc_amount
        previous_position = current_position
        previous_buy_amount = current_buy_amount

        btc_values = []
        position_values = []  

        for match in re.finditer(r'(\d+)[,.]\d+\s+(BTC|BIC)', text, re.IGNORECASE):
            btc_value = match.group(1)
            btc_values.append(int(btc_value))


        for match in re.finditer(r'\b(short|long)\b', text, re.IGNORECASE):
            value = match.group(1).lower()
            if value:  
                position_values.append(value)
        output = ", ".join(position_values)

        print(f"Position Values: [{output}]")
        print("BTC Values:", btc_values)
        
        first_btc_value = None
        second_btc_value = None
        first_position = None
        second_position = None
        third_btc_value = None
        third_position = None

        first_btc_value = btc_values[0] if btc_values else None

        if len(btc_values) > 1:
            second_btc_value = btc_values[1]
        else:
            second_btc_value = None
            
        first_position = position_values[0] if position_values else None
        
        if len(position_values) > 1:
            second_position_value = position_values[1]
        
        if len(position_values) > 2:
            third_position_values = None
        
        if len(btc_values) > 2:
            third_btc_values = None

        
        current_buy_amount = first_btc_value / 100
        current_btc_amount = first_btc_value
        previous_btc_amount = previous_btc_amount
        diff_btc = current_btc_amount - previous_btc_amount
        if diff_btc < 0:
            diff_btcc = (previous_btc_amount - current_btc_amount) / 1000
            diff_btcc = diff_btcc
            
        diff_btc_buy = (current_btc_amount - previous_btc_amount) / 1000
        plus_btc_buy = (current_btc_amount + previous_btc_amount) / 1000
        
        if len(position_values) >1:
            if ("long" in position_values[0] and "short" in position_values[1]) or ("short" in position_values[0] and "long" in position_values[1]):
                ssp_buy = (first_btc_value - second_btc_value) / 1000
                pnt_buy = (secod_btc_value - first_btc_value) / 1000

                i_btc_amount = first_btc_value - second_btc_value
                s_btc_amount = secod_btc_value - first_btc_value
                
        #출력에 short이 적히고 전 포지션이 숏이 아니라면
        if "short" in first_position and previous_position.lower() != "short":
            if previous_position.lower() == "long":
                binance.create_market_sell_order(symbol='BTC/USDT', amount=previous_buy_amount)
                time.sleep(2)
                binance.create_market_sell_order(symbol='BTC/USDT', amount=current_buy_amount)
                current_position = "short".lower()
                current_btc_amount = current_btc_amount
            binance.create_market_sell_order(symbol='BTC/USDT', amount=current_buy_amount)
            current_position = "short".lower()
            current_btc_amount = current_btc_amount


            #출력에 long이 적히고 전 포지션이 롱이 아니라면
        if "long" in first_position and previous_position.lower() != "long":
            if previous_position.lower() == "short":
                binance.create_market_buy_order(symbol="BTC/USDT",amount=previous_buy_amount)
                time.sleep(2)
                binance.create_market_buy_order(symbol="BTC/USDT", amount=current_buy_amount)
                current_position = "long".lower()
                current_btc_amount = current_btc_amount
            binance.create_market_buy_order(symbol="BTC/USDT", amount=current_buy_amount)
            current_position = "long".lower()
            current_btc_amount = current_btc_amount

        # 차이가 양수인 경우, 만큼의 수량을 더 매수
        if diff_btc > 0 and previous_position.lower() == "long":
            if "long" in text.lower():
                binance.create_market_buy_order(symbol="BTC/USDT", amount=diff_btc_buy)
                current_position = "long".lower()
                current_btc_amount = current_btc_amount

            elif "short" in text.lower():
                binance.create_market_sell_order(symbol="BTC/USDT", amount=plus_btc_buy)
                current_position = "short".lower()
                current_btc_amount = current_btc_amount

        if diff_btc < 0 and previous_position.lower() == "long":
            if "long" in text.lower():
                binance.create_market_sell_order(symbol="BTC/USDT", amount=diff_btcc)
                current_position = "long".lower()
                current_btc_amount = current_btc_amount
            elif "short" in text.lower():
                binance.create_market_sell_order(symbol="BTC/USDT", amount=plus_btc_buy)
                current_position = "short".lower()
                current_btc_amount = current_btc_amount

        if diff_btc > 0 and previous_position.lower() == "short":
            if "long" in text.lower():
                binance.create_market_buy_order(symbol="BTC/USDT", amount=plus_btc_buy)
                current_position = "long".lower()
                current_btc_amount = current_btc_amount
            elif "short" in text.lower():
                binance.create_market_sell_order(symbol="BTC/USDT", amount=diff_btc_buy)
                current_position = "short".lower()
                current_btc_amount = current_btc_amount

        if diff_btc < 0 and previous_position.lower() == "short":
            if "long" in text.lower():
                binance.create_market_buy_order(symbol="BTC/USDT", amount=plus_btc_buy)
                current_position = "long".lower()
                current_btc_amount = current_btc_amount
            elif "short" in text.lower():
                binance.create_market_buy_order(symbol="BTC/USDT", amount=diff_btcc)
                current_position = "short".lower()
                current_btc_amount = current_btc_amount

        if len(position_values) > 1:
            if ("long" in position_values[0] and "short" in position_values[1]) or ("short" in position_values[0] and "long" in position_values[1]):
                if previous_position.lower() == "long":
                    if first_btc_value > second_btc_value and first_position == "short":
                        binance.create_market_sell_order(symbol="BTC/USDT",amount = previous_buy_amount)#가진거모두
                        time.sleep(2)
                        binance.create_market_sell_order(symbol="BTC/USDT", amount=ssp_buy)
                        current_position = "short".lower()
                        current_btc_amount = i_btc_amount
                    elif first_btc_value < second_btc_value and first_position == "short":
                        binance.create_market_sell_order(symbol="BTC/USDT", amount=previous_buy_amount)
                        time.sleep(2)
                        binance.create_market_sell_order(symbol="BTC/USDT" , amount = pnt_buy)
                        current_position = "short".lower()
                        current_btc_amount = s_btc_amount

                    if first_btc_value > second_btc_value and second_position == "short":
                        binance.create_market_sell_order(symbol="BTC/USDT",amount = previous_buy_amount)#가진거모두
                        time.sleep(2)
                        binance.create_market_sell_order(symbol="BTC/USDT", amount=ssp_buy)
                        current_position = "short".lower()
                        current_btc_amount = i_btc_amount
                    elif first_btc_value < second_btc_value and second_position == "short":
                        binance.create_market_sell_order(symbol="BTC/USDT", amount=previous_buy_amount)
                        time.sleep(2)
                        binance.create_market_sell_order(symbol="BTC/USDT" , amount = pnt_buy)
                        current_position = "short".lower()
                        current_btc_amount = s_btc_amount

                    if first_btc_value > second_btc_value and first_position == "long":
                        binance.create_market_sell_order(symbol="BTC/USDT",amount = previous_buy_amount)#가진거모두
                        time.sleep(2)
                        binance.create_market_buy_order(symbol="BTC/USDT", amount=ssp_buy)
                        current_position = "long".lower()
                        current_btc_amount = i_btc_amount
                    elif first_btc_value < second_btc_value and first_position == "long":
                        binance.create_market_sell_order(symbol="BTC/USDT", amount=previous_buy_amount)
                        time.sleep(2)
                        binance.create_market_buy_order(symbol="BTC/USDT" , amount = pnt_buy)
                        current_position = "long".lower()
                        current_btc_amount = s_btc_amount

                    if first_btc_value > second_btc_value and second_position == "long":
                        binance.create_market_sell_order(symbol="BTC/USDT",amount = previous_buy_amount)#가진거모두
                        time.sleep(2)
                        binance.create_market_buy_order(symbol="BTC/USDT", amount=ssp_buy)
                        current_position = "long".lower()
                        current_btc_amount = i_btc_amount
                    elif first_btc_value < second_btc_value and second_position == "long":
                        binance.create_market_sell_order(symbol="BTC/USDT", amount=previous_buy_amount)
                        time.sleep(2)
                        binance.create_market_buy_order(symbol="BTC/USDT" , amount = pnt_buy)
                        current_position = "long".lower()
                        current_btc_amount = s_btc_amount

                    if first_btc_value == second_btc_value and first_position == "long":
                        binance.create_market_sell_order(symbol="BTC/USDT", amount=previous_buy_amount)
                        current_position = "short".lower()
                        current_btc_amount = 0
                    elif first_btc_value == second_btc_value and first_position == "short":
                        binance.create_market_sell_order(symbol="BTC/USDT", amount=previous_buy_amount)
                        current_position = "short".lower()
                        current_btc_amount = 0

                    if first_btc_value == second_btc_value and second_position == "long":
                        binance.create_market_sell_order(symbol="BTC/USDT", amount=previous_buy_amount)
                        current_position = "short".lower()
                        current_btc_amount = 0
                    elif first_btc_value == second_btc_value and second_position == "short":
                        binance.create_market_sell_order(symbol="BTC/USDT", amount=previous_buy_amount)
                        current_position = "short".lower()
                        current_btc_amount = 0

                if previous_position.lower() == "short":
                    if first_btc_value > second_btc_value and first_position == "short":
                        binance.create_market_buy_order(symbol="BTC/USDT",amount = previous_buy_amount)#가진거모두
                        time.sleep(2)
                        binance.create_market_sell_order(symbol="BTC/USDT", amount=ssp_buy)
                        current_position = "short".lower()
                        current_btc_amount = i_btc_amount
                    elif first_btc_value < second_btc_value and first_position == "short":
                        binance.create_market_buy_order(symbol="BTC/USDT", amount=previous_buy_amount)
                        time.sleep(2)
                        binance.create_market_sell_order(symbol="BTC/USDT" , amount = pnt_buy)
                        current_position = "short".lower()
                        current_btc_amount = s_btc_amount

                    if first_btc_value > second_btc_value and second_position == "short":
                        binance.create_market_buy_order(symbol="BTC/USDT",amount = previous_buy_amount)#가진거모두
                        time.sleep(2)
                        binance.create_market_sell_order(symbol="BTC/USDT", amount=ssp_buy)
                        current_position = "short".lower()
                        current_btc_amount =i_btc_amount

                    elif first_btc_value < second_btc_value and second_position == "short":
                        binance.create_market_buy_order(symbol="BTC/USDT", amount=previous_buy_amount)
                        time.sleep(2)
                        binance.create_market_sell_order(symbol="BTC/USDT" , amount = pnt_buy)
                        current_position = "short".lower()
                        current_btc_amount = s_btc_amount

                    if first_btc_value > second_btc_value and first_position == "long":
                        binance.create_market_buy_order(symbol="BTC/USDT",amount = previous_buy_amount)#가진거모두
                        time.sleep(2)
                        binance.create_market_buy_order(symbol="BTC/USDT", amount=ssp_buy)
                        current_position = "long".lower()
                        current_btc_amount =i_btc_amount
                    elif first_btc_value < second_btc_value and first_position == "long":
                        binance.create_market_buy_order(symbol="BTC/USDT", amount=previous_buy_amount)
                        time.sleep(2)
                        binance.create_market_sell_order(symbol="BTC/USDT" , amount = pnt_buy)
                        current_position = "long".lower()
                        current_btc_amount = s_btc_amount

                    if first_btc_value > second_btc_value and second_position == "long":
                        binance.create_market_buy_order(symbol="BTC/USDT",amount = previous_buy_amount)#가진거모두
                        time.sleep(2)
                        binance.create_market_buy_order(symbol="BTC/USDT", amount=ssp_buy)
                        current_position = "long".lower()
                        current_btc_amount = i_btc_amount
                    elif first_btc_value < second_btc_value and second_position == "long":
                        binance.create_market_buy_order(symbol="BTC/USDT", amount=previous_buy_amount)
                        time.sleep(2)
                        binance.create_market_buy_order(symbol="BTC/USDT" , amount = pnt_buy)
                        current_position = "long".lower()
                        current_btc_amount = s_btc_amount

                    if first_btc_value == second_btc_value and first_position == "long":
                        binance.create_market_buy_order(symbol="BTC/USDT", amount=previous_buy_amount)
                        current_position = "long".lower()
                        current_btc_amount = 0
                    elif first_btc_value == second_btc_value and first_position == "short":
                        binance.create_market_buy_order(symbol="BTC/USDT", amount=previous_buy_amount)
                        current_position = "long".lower()
                        current_btc_amount =0

                    if first_btc_value == second_btc_value and second_position == "long":
                        binance.create_market_buy_order(symbol="BTC/USDT", amount=previous_buy_amount)
                        current_position = "long".lower()
                        current_btc_amount = 0
                    elif first_btc_value == second_btc_value and second_position == "short":
                        binance.create_market_buy_order(symbol="BTC/USDT", amount=previous_buy_amount)
                        current_position = "long".lower()
                        current_btc_amount = 0
                        
            if len(btc_values) < 0:
                if previous_position == "short": 
                    binance.create_market_buy_order(symbol ="BTC/USDT" , amount = previous_buy_amount)
            if len(btc_values) < 0:
                if previous_position == "long":
                    binance.create_market_sell_order(symbol ="BTC/USDT" , amount = previous_buy_amount)
                    
    print("현재 포지션 :" ,current_position)
    print("전 btc수량" , previous_btc_amount)
    print("지금 btc 수량" , current_btc_amount)
        
    time.sleep(8)